# AI Detektor Vremena

## Kreiranje modela za detekciju vremena sa slike

importi za ucitavanje

In [1]:
import pandas

import numpy as np
import matplotlib.pyplot as plt
import cv2

Definisanje globalnih promenljivih

In [2]:
PUTANJA = 'C:/Users/User/Desktop/AI'# promeniti na svoju lokaciju foldera koji sadrzi test folder, train folder, test_labels.csv i train_labels.csv
KATEGORIJE = ['sunrise', 'cloudy', 'shine', 'rain']
IMG_SIZE = 400

Pravljenje niza labela "train_y" i punimo ga brojevnim reprezentacijama zadatih kategorija (napravljn da prihvata pogresne i podatke)

In [3]:
csv_fajl = pandas.read_csv(PUTANJA + r"\train_labels.csv")

train_y = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['weather'] == 'sunrise':
        train_y = np.append(train_y, 0)
    elif row['weather'] == 'cloudy':
        train_y = np.append(train_y, 1)
    elif row['weather'] == 'shine':
        train_y = np.append(train_y, 2)
    elif row['weather'] == 'rain':
        train_y = np.append(train_y, 3)
    else:
        print("Neodgovarajuca labela na slici {}".format(row['weather']))
        
trainy2 = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['weather'] == 'sunrise':
        trainy2 = np.append(trainy2, 0)
    elif row['weather'] == 'cloudy':
        trainy2 = np.append(trainy2, 1)
    elif row['weather'] == 'shine':
        trainy2 = np.append(trainy2, 2)
    elif row['weather'] == 'rain':
        trainy2 = np.append(trainy2, 3)
    else:
        trainy2 = np.append(trainy2, -1)

Ucitavanje slika

In [4]:
csv_fajl.head()
training_data = []

for index, row in csv_fajl.iterrows():
    if trainy2[index] != -1:
        img_name = row['file_name']
        lokacija = PUTANJA + '/train/' + str(img_name)
        try:# za slucaj da neka slika ostecena
            slika = cv2.imread(lokacija)
            slika = cv2.cvtColor(slika, cv2.COLOR_BGR2GRAY)
            slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
            training_data.append([slika, int(trainy2[index])])
        except:
            print("Ostecena slika " + str(img_name))

## HOG

In [5]:
featuresList = []
labels = []

nbins = 9 # broj binova
cell_size = (8, 8)# broj piksela po celiji (moze da se poveca za brzi rad)
block_size = (3, 3) # broj celija po bloku

hog = cv2.HOGDescriptor(_winSize=(IMG_SIZE // cell_size[1] * cell_size[1], 
                                  IMG_SIZE // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)


for features, label in training_data:
    featuresList.append(hog.compute(features))
    labels.append(label)

print(featuresList[0].shape)

X = np.array(featuresList)
y = np.array(labels)

(186624, 1)


Oblikujemo nizove

In [6]:
nsamples, nx, ny = X.shape
X = X.reshape((nsamples, nx*ny))

## Cuvanje podatke koje smo ucitali (da ne bi smo morali da ih ucitavamo svaki put)

In [7]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

## Kreiranje Modela

Ucitavamo prethodno ucitane podatke (po potrebi)

In [8]:
import pickle
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

## SVC

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel="linear", decision_function_shape='ovr')
svm.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Testiranje modela

In [10]:
test_csv_fajl = pandas.read_csv(PUTANJA + r"\test_labels.csv")

test_y = np.ndarray(0)

for index, row in test_csv_fajl.iterrows():
    if row['weather'] == 'sunrise':
        test_y = np.append(test_y, 0)
    elif row['weather'] == 'cloudy':
        test_y = np.append(test_y, 1)
    elif row['weather'] == 'shine':
        test_y = np.append(test_y, 2)
    elif row['weather'] == 'rain':
        test_y = np.append(test_y, 3)
    else:
        print("Neodgovarajuca labela na slici {}".format(row['weather']))
        
test_trainy2 = np.ndarray(0)

for index, row in test_csv_fajl.iterrows():
    if row['weather'] == 'sunrise':
        test_trainy2 = np.append(test_trainy2, 0)
    elif row['weather'] == 'cloudy':
        test_trainy2 = np.append(test_trainy2, 1)
    elif row['weather'] == 'shine':
        test_trainy2 = np.append(test_trainy2, 2)
    elif row['weather'] == 'rain':
        test_trainy2 = np.append(test_trainy2, 3)
    else:
        test_trainy2 = np.append(test_trainy2, -1)

In [11]:
test_csv_fajl.head()
test_training_data = []

for index, row in test_csv_fajl.iterrows():
    if test_trainy2[index] != -1:
        img_name = row['file_name']
        lokacija = PUTANJA + '/test/' + str(img_name)
        try:
            slika = cv2.imread(lokacija)
            slika = cv2.cvtColor(slika, cv2.COLOR_BGR2GRAY)
            slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
            test_training_data.append([slika, int(test_trainy2[index])])
        except:
            print("Ostecena slika " + str(img_name))

In [12]:
test_featuresList = []
test_labels = []

for test_features, test_label in test_training_data:
    test_featuresList.append(hog.compute(test_features))
    test_labels.append(test_label)

print(test_featuresList[0].shape)

test_X = np.array(test_featuresList)
test_y = np.array(test_labels)

(186624, 1)


In [13]:
nsamples, nx, ny = test_X.shape
test_X = test_X.reshape((nsamples, nx*ny))

print(np.shape(test_X), np.shape(test_y))

(225, 186624) (225,)


In [14]:
y3 = svm.predict(test_X)
print("score = {} (trazi se bar 0.7 preciznost)".format(accuracy_score(test_y, y3)))

score = 0.7911111111111111 (trazi se bar 0.7 preciznost)
